# comparar variando ventanas de tiempo

## Packages

In [32]:
from datetime import datetime, timedelta
import pandas as pd
import os
from sunpy.net import Fido, attrs as a
from sunpy.timeseries import TimeSeries
from sunpy.timeseries.sources.goes import XRSTimeSeries
import astropy.units as u
from sunkit_instruments.goes_xrs import calculate_temperature_em
import matplotlib.pyplot as plt
import numpy as np
from sunpy.data import manager
import netCDF4 as nc
import os
import pandas as pd
from datetime import datetime, timedelta
import copy
from matplotlib import colormaps
list(colormaps)
#from colorspacious import cspace_converter
import matplotlib as mpl
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import cm
from matplotlib.colors import ListedColormap
import matplotlib.dates as mdates
from matplotlib.ticker import LogFormatter
from matplotlib.ticker import LogFormatterMathtext
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FuncFormatter
from IPython.display import HTML, display
import matplotlib.pyplot as plt
import seaborn as sns
import re
import glob

## Carpeta para csv de diferentes windows

In [33]:
n=1230
# Obtener fecha actual en formato YYYY-MM-DD
#fecha_actual = datetime.now().strftime("%Y-%m-%d")
fecha_actual = "2025-10-24"
window_minutes=30
output_dir = f"{fecha_actual}_Analysis_for_{n}_days/Analysis"

# Crear subcarpeta para guardar resultados de análisis
analysis_dir = os.path.join(output_dir, "WindowsFAI")
# Crear subcarpeta para guardar resultados de análisis (si no existe) y mostrar mensaje
if os.path.exists(analysis_dir):
    print(f"⚠️ La carpeta de análisis ya existía: {analysis_dir}")
else:
    os.makedirs(analysis_dir , exist_ok=True)
    print(f"📁 Carpeta de análisis creada: {analysis_dir}")





⚠️ La carpeta de análisis ya existía: 2025-10-24_Analysis_for_1230_days/Analysis/WindowsFAI


In [34]:
# Path of cleaned data in csv
csv_path_full_valid = f"{fecha_actual}_Analysis_for_{n}_days/df_full_{n}_valid.csv"
# Path of valid flares: 
csv_path_flares = f"{fecha_actual}_Analysis_for_{n}_days/df_flares_{n}_valid.csv"
# Path of anticipation_time
csv_path_anticipation_time = f"{fecha_actual}_Analysis_for_{n}_days/Analysis/df_anticipation_time(W_{window_minutes}).csv"
# Path of fai_selected
csv_path_fai_selected = f"{fecha_actual}_Analysis_for_{n}_days/Analysis/df_fai_selected.csv"
# Path of df_full
csv_path_full = f"{fecha_actual}_Analysis_for_{n}_days/df_full_goes_{n}.csv"
# Path of df_full
csv_path_fai_assoc = f"{fecha_actual}_Analysis_for_{n}_days/Analysis/df_fai_assoc.csv"

df_full_valid = pd.read_csv(csv_path_full_valid)
df_flares_valid = pd.read_csv(csv_path_flares)
df_anticipation_time = pd.read_csv(csv_path_anticipation_time)
df_fai_selected = pd.read_csv(csv_path_fai_selected)
df_full = pd.read_csv(csv_path_full)
df_fai_assoc = pd.read_csv(csv_path_fai_assoc)

## Función para anticipation_fai_analysis_v2

In [35]:
######################################
# 8. Tiempo de Anticipacion de flare #
######################################

def anticipation_fai_analysis_v2(df_fai_selected, df_flare_data,
                                start_col="StartTime",
                                peak_col="PeakTime",
                                end_col="EndTime",
                                window_minutes=30, # minutos antes del flare
                                max_prev_flare_minutes=180): # rango para calcular tiempo desde flare anterior
    """
    Analiza FAIs alrededor de cada flare y calcula varias métricas:
    - cantidad de FAIs en distintas ventanas temporales
    - tiempos de anticipación respecto a StartTime y PeakTime
    - tiempo desde el flare anterior (si está dentro del rango)
    Parámetros:
    - df_fai_selected: DataFrame con FAIs (índice temporal)
    - df_flare_data: DataFrame con columnas StartTime, PeakTime, EndTime
    - window_minutes: minutos antes del flare a considerar para buscar FAIs
    - max_prev_flare_minutes: rango máximo para calcular tiempo desde flare anterior

    Retorna:
    - df_flare_data con columnas nuevas:
        FAIalerts_W, FAIalerts_WStart,FAIalerts_WPeak, FAIalerts_StartPeak, FAIalerts_PeakEnd,
        AnticipationStart, AnticipationPeak, Peak_to_lastFAI, Time_since_prev_flare
    """

    df = df_flare_data.copy()
    df[start_col] = pd.to_datetime(df[start_col])
    df[peak_col] = pd.to_datetime(df[peak_col])
    df[end_col] = pd.to_datetime(df[end_col])
    fai_times = pd.to_datetime(df_fai_selected["date"]).sort_values()

    # asegurar orden temporal
    df = df.sort_values(peak_col).reset_index(drop=True)

    # columnas a crear
    results = {
        "FAIalerts_W": [],
        "FAIalerts_WStart": [],
        "FAIalerts_WPeak": [],
        "FAIalerts_StartPeak": [],
        "FAIalerts_PeakEnd": [],
        "AnticipationStart": [],
        "AnticipationPeak": [],
        "Peak_to_lastFAI": [],
        "Time_since_prev_flare": []
    }

    for i, row in df.iterrows():
        start_t = row[start_col]
        peak_t = row[peak_col]
        end_t = row[end_col]

        # 🔹 Límite inferior: EndTime del flare anterior o ventana
        if i > 0:
            prev_end = df.loc[i-1, end_col]
        else:
            prev_end = pd.Timestamp.min

        lower_limit = max(prev_end, peak_t - timedelta(minutes=window_minutes))
        upper_limit = end_t

        # 🔹 FAIs válidos en la ventana general
        valid_fais = fai_times[(fai_times >= lower_limit) & (fai_times <= upper_limit)]

        # Contar FAIs por zonas
        w_total = len(valid_fais)
        w_start = len(valid_fais[valid_fais < start_t])
        w_peak = len(valid_fais[valid_fais < peak_t])
        start_peak = len(valid_fais[(valid_fais >= start_t) & (valid_fais < peak_t)])
        peak_end = len(valid_fais[(valid_fais >= peak_t) & (valid_fais <= end_t)])

        results["FAIalerts_W"].append(w_total)
        results["FAIalerts_WStart"].append(w_start)
        results["FAIalerts_WPeak"].append(w_peak)
        results["FAIalerts_StartPeak"].append(start_peak)
        results["FAIalerts_PeakEnd"].append(peak_end)

        # 🔹 Calcular anticipaciones
        if not valid_fais.empty:
            earliest_fai = valid_fais.min()
            last_fai = valid_fais.max()

            anticipation_start = (start_t - earliest_fai).total_seconds() / 60.0 if earliest_fai < start_t else None
            anticipation_peak = (peak_t - earliest_fai).total_seconds() / 60.0 if earliest_fai < peak_t else None
            peak_to_last_fai = (peak_t - last_fai).total_seconds() / 60.0  # puede ser negativo

        else:
            anticipation_start = anticipation_peak = peak_to_last_fai = None

        results["AnticipationStart"].append(anticipation_start)
        results["AnticipationPeak"].append(anticipation_peak)
        results["Peak_to_lastFAI"].append(peak_to_last_fai)

        # 🔹 Calcular tiempo desde el flare anterior
        
        if i > 0:
            prev_flare_time = df.loc[i-1, end_col]
            delta_min = (peak_t - prev_flare_time).total_seconds() / 60.0
            time_since_prev = delta_min if delta_min <= max_prev_flare_minutes else None
        else:
            time_since_prev = None

        results["Time_since_prev_flare"].append(time_since_prev)

    # Añadir las nuevas columnas al dataframe
    for col, values in results.items():
        df[col] = values

    return df



In [36]:
#df_anticipation_time
def run_anticipation_analysis_multiple_windows(df_fai_selected, df_flares_valid, 
                                             window_start=30, window_end=60, step=5,
                                             start_col="StartTime", peak_col="PeakTime", 
                                             end_col="EndTime", max_prev_flare_minutes=180):
    """
    Ejecuta anticipation_fai_analysis_v2 para múltiples ventanas de tiempo.
    """
    
    # Crear directorio para resultados
    anticipation_dir = os.path.join(analysis_dir, f"anticipation_analysis_{window_start}to{window_end}_step{step}")
    os.makedirs(anticipation_dir, exist_ok=True)
    
    windows = range(window_start, window_end + 1, step)
    
    print(f"🔍 Iniciando ANÁLISIS DE ANTICIPACIÓN para {len(windows)} ventanas:")
    print(f"   Rango: {window_start}-{window_end} min, Paso: {step} min")
    print(f"   Ventanas: {list(windows)}")
    print("-" * 60)
    
    results_summary = []
    
    for window_minutes in windows:
        print(f"📊 Procesando anticipation analysis con ventana de {window_minutes} minutos...")
        
        try:
            # Ejecutar análisis de anticipación
            df_anticipation = anticipation_fai_analysis_v2(
                df_fai_selected=df_fai_selected,
                df_flare_data=df_flares_valid,
                start_col=start_col,
                peak_col=peak_col,
                end_col=end_col,
                window_minutes=window_minutes,
                max_prev_flare_minutes=max_prev_flare_minutes
            )
            
            # Calcular estadísticas
            total_flares = len(df_anticipation)
            flares_with_fai = (df_anticipation['FAIalerts_W'] > 0).sum()
            anticipation_rate = (flares_with_fai / total_flares * 100) if total_flares > 0 else 0
            
            # Guardar dataframe
            filename = f"df_anticipation_{window_minutes}min.csv"
            filepath = os.path.join(anticipation_dir, filename)
            df_anticipation.to_csv(filepath, index=False)
            
            # Guardar estadísticas
            stats = {
                'window_minutes': window_minutes,
                'total_flares': total_flares,
                'flares_with_fai': flares_with_fai,
                'anticipation_rate': anticipation_rate,
                'filename': filename
            }
            results_summary.append(stats)
            
            print(f"   ✅ {flares_with_fai}/{total_flares} flares con FAI ({anticipation_rate:.1f}%)")
            print(f"   💾 Guardado como: {filename}")
            
        except Exception as e:
            print(f"   ❌ Error en ventana {window_minutes}min: {e}")
            continue
    
    # Guardar resumen de resultados
    df_summary = pd.DataFrame(results_summary)
    summary_path = os.path.join(anticipation_dir, f"anticipation_summary_{window_start}to{window_end}_step{step}.csv")
    df_summary.to_csv(summary_path, index=False)
    
    print("-" * 60)
    print(f"🎯 ANÁLISIS DE ANTICIPACIÓN COMPLETADO:")
    print(f"   📁 Resultados guardados en: {anticipation_dir}")
    print(f"   📊 Resumen guardado en: {summary_path}")
    
    # Mostrar resumen final
    print(f"\n📈 RESUMEN ANTICIPACIÓN:")
    print(f"{'Window':<10} {'Total Flares':<14} {'With FAI':<12} {'Rate (%)':<10}")
    print("-" * 50)
    for stats in results_summary:
        print(f"{stats['window_minutes']:<10} {stats['total_flares']:<14} {stats['flares_with_fai']:<12} {stats['anticipation_rate']:<10.1f}")
    
    return df_summary, anticipation_dir

In [37]:
# Ejecutar SOLO análisis de anticipación
'''
df_anticipation_summary, anticipation_dir = run_anticipation_analysis_multiple_windows(
    df_fai_selected=df_fai_selected,
    df_flares_valid=df_flares_valid,
    window_start=30,
    window_end=60,
    step=5,
    max_prev_flare_minutes=180
)
'''

'\ndf_anticipation_summary, anticipation_dir = run_anticipation_analysis_multiple_windows(\n    df_fai_selected=df_fai_selected,\n    df_flares_valid=df_flares_valid,\n    window_start=30,\n    window_end=60,\n    step=5,\n    max_prev_flare_minutes=180\n)\n'

## Función para associate_fai_to_flare_dataframes

In [38]:
def associate_fai_to_flare_dataframes(df_fai_selected, df_flares, window_minutes=30, time_ref='PeakTime'):
    """
    Asocia cada alerta FAI a un flare que ocurra dentro de una ventana hacia adelante.

    Parameters:
    - df_fai_selected: DataFrame con alertas FAI (debe tener columna 'date' o similar)
    - df_flares: DataFrame con flares
    - window_minutes: ventana de búsqueda en minutos (hacia adelante desde la FAI)
    - time_ref: 'PeakTime' o 'StartTime', qué hora del flare usar para medir distancia

    Returns:
    - df_fai: DataFrame de FAIs con columnas adicionales sobre el flare asociado
    """
    # Crear copias para no modificar los originales
    df_fai = df_fai_selected.copy()
    df_flares_copy = df_flares.copy()

    # Identificar la columna de tiempo en df_fai_selected
    time_col_fai = None
    for col in ['date', 'Time_FAI', 'Unnamed: 0']:
        if col in df_fai.columns:
            time_col_fai = col
            break
    
    if time_col_fai is None:
        # Si no encuentra una columna conocida, usar la primera columna que parezca datetime
        for col in df_fai.columns:
            if df_fai[col].dtype == 'object':
                try:
                    pd.to_datetime(df_fai[col].iloc[0])
                    time_col_fai = col
                    break
                except:
                    continue
    
    if time_col_fai is None:
        raise ValueError("No se pudo identificar la columna de tiempo en df_fai_selected")

    print(f"Usando columna de tiempo FAI: {time_col_fai}")

    # Convertir a datetime
    df_fai['Time_FAI'] = pd.to_datetime(df_fai[time_col_fai])

    # Asegurar formato datetime en flares
    df_flares_copy['StartTime'] = pd.to_datetime(df_flares_copy['StartTime'])
    df_flares_copy['EndTime'] = pd.to_datetime(df_flares_copy['EndTime'])
    df_flares_copy['PeakTime'] = pd.to_datetime(df_flares_copy['PeakTime'])

    # Inicializar columnas
    df_fai['Associated_Flare'] = False
    df_fai['Flare_StartTime'] = pd.NaT
    df_fai['Flare_EndTime'] = pd.NaT
    df_fai['Flare_PeakTime'] = pd.NaT
    df_fai['Flare_Class'] = None
    df_fai['Time_to_flare'] = None
    df_fai['Flare_Observatory'] = None

    window = pd.Timedelta(minutes=window_minutes)

    print(f"Procesando {len(df_fai)} alertas FAI...")
    
    associated_count = 0

    for idx, row in df_fai.iterrows():
        fai_time = row['Time_FAI']

        # Flares dentro de la ventana hacia adelante
        mask = (df_flares_copy[time_ref] >= fai_time) & (df_flares_copy[time_ref] <= fai_time + window)
        candidate_flares = df_flares_copy[mask].sort_values(time_ref)

        if not candidate_flares.empty:
            flare = candidate_flares.iloc[0]  # Tomamos el primero más cercano
            df_fai.at[idx, 'Associated_Flare'] = True
            df_fai.at[idx, 'Flare_StartTime'] = flare['StartTime']
            df_fai.at[idx, 'Flare_EndTime'] = flare['EndTime']
            df_fai.at[idx, 'Flare_PeakTime'] = flare['PeakTime']
            df_fai.at[idx, 'Flare_Class'] = flare['Class']
            df_fai.at[idx, 'Time_to_flare'] = (flare[time_ref] - fai_time).total_seconds() / 60  # minutos
            
            # Añadir observatorio si está disponible
            if 'Observatory' in flare:
                df_fai.at[idx, 'Flare_Observatory'] = flare['Observatory']
            
            associated_count += 1

    # Estadísticas
    total_fai = len(df_fai)
    percentage = (associated_count / total_fai) * 100
    
    print(f"\n--- Estadísticas de Asociación FAI-Flare ---")
    print(f"Total alertas FAI: {total_fai}")
    print(f"Alertas FAI asociadas a flares: {associated_count} ({percentage:.1f}%)")
    print(f"Ventana utilizada: {window_minutes} minutos")
    print(f"Tiempo de referencia: {time_ref}")
    
    if associated_count > 0:
        avg_time_to_flare = df_fai['Time_to_flare'].dropna().mean()
        min_time_to_flare = df_fai['Time_to_flare'].dropna().min()
        max_time_to_flare = df_fai['Time_to_flare'].dropna().max()
        print(f"Tiempo promedio a flare: {avg_time_to_flare:.1f} minutos")
        print(f"Tiempo mínimo a flare: {min_time_to_flare:.1f} minutos")
        print(f"Tiempo máximo a flare: {max_time_to_flare:.1f} minutos")
        
        # Distribución por clase de flare
        class_dist = df_fai[df_fai['Associated_Flare']]['Flare_Class'].value_counts()
        print(f"\nDistribución por clase de flare:")
        for cls, count in class_dist.items():
            print(f"  {cls}: {count}")

    return df_fai



In [39]:
def run_association_analysis_multiple_windows(df_fai_selected, df_flares_valid,
                                            window_start=30, window_end=60, step=5,
                                            time_ref='PeakTime'):
    """
    Ejecuta associate_fai_to_flare_dataframes para múltiples ventanas de tiempo.
    """
    
    # Crear directorio para resultados
    association_dir = os.path.join(analysis_dir, f"association_analysis_{window_start}to{window_end}_step{step}")
    os.makedirs(association_dir, exist_ok=True)
    
    windows = range(window_start, window_end + 1, step)
    
    print(f"🔍 Iniciando ANÁLISIS DE ASOCIACIÓN para {len(windows)} ventanas:")
    print(f"   Rango: {window_start}-{window_end} min, Paso: {step} min")
    print(f"   Ventanas: {list(windows)}")
    print(f"   Referencia temporal: {time_ref}")
    print("-" * 60)
    
    results_summary = []
    
    for window_minutes in windows:
        print(f"📊 Procesando association analysis con ventana de {window_minutes} minutos...")
        
        try:
            # Ejecutar análisis de asociación
            df_fai_assoc = associate_fai_to_flare_dataframes(
                df_fai_selected=df_fai_selected,
                df_flares=df_flares_valid,
                window_minutes=window_minutes,
                time_ref=time_ref
            )
            
            # Calcular estadísticas
            total_fais = len(df_fai_assoc)
            associated_fais = df_fai_assoc['Associated_Flare'].sum()
            association_rate = (associated_fais / total_fais * 100) if total_fais > 0 else 0
            
            # Guardar dataframe
            filename = f"df_fai_assoc_{window_minutes}min.csv"
            filepath = os.path.join(association_dir, filename)
            df_fai_assoc.to_csv(filepath, index=False)
            
            # Guardar estadísticas
            stats = {
                'window_minutes': window_minutes,
                'total_fais': total_fais,
                'associated_fais': associated_fais,
                'association_rate': association_rate,
                'filename': filename
            }
            results_summary.append(stats)
            
            print(f"   ✅ {associated_fais}/{total_fais} FAIs asociados ({association_rate:.1f}%)")
            print(f"   💾 Guardado como: {filename}")
            
        except Exception as e:
            print(f"   ❌ Error en ventana {window_minutes}min: {e}")
            continue
    
    # Guardar resumen de resultados
    df_summary = pd.DataFrame(results_summary)
    summary_path = os.path.join(association_dir, f"association_summary_{window_start}to{window_end}_step{step}.csv")
    df_summary.to_csv(summary_path, index=False)
    
    print("-" * 60)
    print(f"🎯 ANÁLISIS DE ASOCIACIÓN COMPLETADO:")
    print(f"   📁 Resultados guardados en: {association_dir}")
    print(f"   📊 Resumen guardado en: {summary_path}")
    
    # Mostrar resumen final
    print(f"\n📈 RESUMEN ASOCIACIÓN:")
    print(f"{'Window':<10} {'Total FAIs':<12} {'Associated':<12} {'Rate (%)':<10}")
    print("-" * 50)
    for stats in results_summary:
        print(f"{stats['window_minutes']:<10} {stats['total_fais']:<12} {stats['associated_fais']:<12} {stats['association_rate']:<10.1f}")
    
    return df_summary, association_dir

In [40]:
# Ejecutar SOLO análisis de asociación
'''
df_association_summary, association_dir = run_association_analysis_multiple_windows(
    df_fai_selected=df_fai_selected,
    df_flares_valid=df_flares_valid,
    window_start=30,
    window_end=60,
    step=5,
    time_ref='PeakTime'
)
'''

"\ndf_association_summary, association_dir = run_association_analysis_multiple_windows(\n    df_fai_selected=df_fai_selected,\n    df_flares_valid=df_flares_valid,\n    window_start=30,\n    window_end=60,\n    step=5,\n    time_ref='PeakTime'\n)\n"

## Función para ejecutar AMBOS análisis

In [41]:
def run_both_analyses_multiple_windows(df_fai_selected, df_flares_valid,
                                      window_start=30, window_end=60, step=5,
                                      time_ref='PeakTime', max_prev_flare_minutes=180):
    """
    Ejecuta ambos análisis (anticipación y asociación) para múltiples ventanas.
    """
    
    print("🚀 INICIANDO ANÁLISIS COMPLETO")
    print("=" * 60)
    
    # Ejecutar análisis de anticipación
    print("\n1. EJECUTANDO ANÁLISIS DE ANTICIPACIÓN...")
    df_anticipation_summary, anticipation_dir = run_anticipation_analysis_multiple_windows(
        df_fai_selected=df_fai_selected,
        df_flares_valid=df_flares_valid,
        window_start=window_start,
        window_end=window_end,
        step=step,
        max_prev_flare_minutes=max_prev_flare_minutes
    )
    
    # Ejecutar análisis de asociación
    print("\n2. EJECUTANDO ANÁLISIS DE ASOCIACIÓN...")
    df_association_summary, association_dir = run_association_analysis_multiple_windows(
        df_fai_selected=df_fai_selected,
        df_flares_valid=df_flares_valid,
        window_start=window_start,
        window_end=window_end,
        step=step,
        time_ref=time_ref
    )
    
    print("\n" + "=" * 60)
    print("🎯 ANÁLISIS COMPLETO FINALIZADO")
    print(f"   📁 Anticipación: {anticipation_dir}")
    print(f"   📁 Asociación: {association_dir}")
    
    return df_anticipation_summary, df_association_summary, anticipation_dir, association_dir

In [42]:
# Ejecutar AMBOS análisis juntos
df_anticipation_summary, df_association_summary, anticipation_dir, association_dir = run_both_analyses_multiple_windows(
    df_fai_selected=df_fai_selected,
    df_flares_valid=df_flares_valid,
    window_start=65,
    window_end=120,
    step=5,
    time_ref='PeakTime',
    max_prev_flare_minutes=180
)

🚀 INICIANDO ANÁLISIS COMPLETO

1. EJECUTANDO ANÁLISIS DE ANTICIPACIÓN...
🔍 Iniciando ANÁLISIS DE ANTICIPACIÓN para 12 ventanas:
   Rango: 65-120 min, Paso: 5 min
   Ventanas: [65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120]
------------------------------------------------------------
📊 Procesando anticipation analysis con ventana de 65 minutos...


   ✅ 6990/7787 flares con FAI (89.8%)
   💾 Guardado como: df_anticipation_65min.csv
📊 Procesando anticipation analysis con ventana de 70 minutos...
   ✅ 6992/7787 flares con FAI (89.8%)
   💾 Guardado como: df_anticipation_70min.csv
📊 Procesando anticipation analysis con ventana de 75 minutos...
   ✅ 6994/7787 flares con FAI (89.8%)
   💾 Guardado como: df_anticipation_75min.csv
📊 Procesando anticipation analysis con ventana de 80 minutos...
   ✅ 6997/7787 flares con FAI (89.9%)
   💾 Guardado como: df_anticipation_80min.csv
📊 Procesando anticipation analysis con ventana de 85 minutos...
   ✅ 7001/7787 flares con FAI (89.9%)
   💾 Guardado como: df_anticipation_85min.csv
📊 Procesando anticipation analysis con ventana de 90 minutos...
   ✅ 7002/7787 flares con FAI (89.9%)
   💾 Guardado como: df_anticipation_90min.csv
📊 Procesando anticipation analysis con ventana de 95 minutos...
   ✅ 7004/7787 flares con FAI (89.9%)
   💾 Guardado como: df_anticipation_95min.csv
📊 Procesando anticipation an